In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
np.random.seed(42)
import os
import pandas as pd

from utils.loading import TICKER_PATHS, BUYER_PATH, SELLER_PATH
from utils.loading import read_ticker
from utils.preprocessing import preprocess_ticker
from utils.processing import buyer_seller_stats

In [ ]:
import pandas as pd
import numpy as np

num_bins = 250
volume_bins = pd.qcut(df3['volume_pct'], q=num_bins, duplicates='drop')
binned_data = df3.groupby(volume_bins)['price_impact_pct'].mean().reset_index(name='avg_price_impact')

In [ ]:
binned_data['volume_pct'] = binned_data['volume_pct'].apply(lambda x: x.right)

In [ ]:
# plot price impact vs volume
# df_trader_stats = df_trader_stats[df_trader_stats['price_impact'] > 0]
# draw a square root line
x = np.logspace(-3, 1, 1000)
# create sqrt line with intercept 0.1
y_sqrt = np.sqrt(x)
y_linear = x

plt.loglog(x, y_sqrt, label='sqrt')
plt.loglog(x, y_linear, label='linear')
plt.legend()

sns.scatterplot(data=binned_data, x='volume_pct', y='avg_price_impact', alpha=0.5)
plt.xlabel('Total Volume')
plt.ylabel('Price Impact')
plt.xscale('log')
plt.yscale('log')
plt.title('Price Impact vs Total Volume')
plt.show()

In [ ]:
0.1 + np.sqrt(x), np.sqrt(x)

In [ ]:
trader_stats

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Assuming df is your preprocessed DataFrame

# Aggregate data
aggregated_data = df_cleaned.resample('min').agg({
    'trade-volume': 'sum',
    'trade-price': lambda x: np.average(x, weights=df_cleaned.loc[x.index, 'trade-volume'])  # Weighted average
})

# Calculate price changes
aggregated_data['price_change'] = aggregated_data['trade-price'].diff().abs()

# Square root of trade volume
aggregated_data['sqrt_volume'] = np.sqrt(aggregated_data['trade-volume'])

# Linear regression
X = sm.add_constant(aggregated_data['sqrt_volume'])  # Add constant term
y = aggregated_data['price_change']
model = sm.OLS(y, X, missing='drop').fit()  # Drop NaN values

# Plotting
plt.scatter(aggregated_data['sqrt_volume'], aggregated_data['price_change'])
plt.plot(aggregated_data['sqrt_volume'], model.predict(X), color='red')  # Regression line
plt.xlabel('Square Root of Trade Volume')
plt.ylabel('Absolute Price Change')
plt.show()

# Output model summary
print(model.summary())
